In [ ]:
import os

id = 99
pwd = os.getcwd()
print pwd

new_id = '12345678' + str(id)
new_id = new_id[-4:]
print new_id

In [ ]:
#Submit script!!!
import json
import re
import pandas as pd
from finalize_JL import find_neighbors
from finalize_JL import read_mutations_resfile
import subprocess

#Parses dataset .json file and outputs chain to move and input PDB file directory
def json_parser():
    
    asdf = open("/Users/jameslucas/Kortemme_Rotation/blank_job_dict_copy.json")
    jsonfile = json.load(asdf)
    
    key = sorted(jsonfile.keys())[0]
    chaintomove = jsonfile[key]["%%chainstomove%%"]
    directory = jsonfile[key]['input_file_list'][0]
    
    return chaintomove, directory

#Finds neighbors within 8A and adds position and Chain information to a pandas dataframe
def neighbors_list(pdb_filepath, pdb_file):
    neighbors = find_neighbors("/Users/jameslucas/Kortemme_Rotation/"+pdb_filepath, "/Users/jameslucas/Kortemme_Rotation/"+pdb_file, 8)
    pivotlist = ''
    for i in neighbors:
        string_parse = re.sub("[(),']",'', str(i))
        for s in string_parse.split():
            if s.isdigit():
                pivotlist = pivotlist + s
            else:
                pivotlist = pivotlist + s + ','
                
    pivotlist = pivotlist[:-1]
    return pivotlist

#Reads resfile and returns mutation position+chain and type
def resfile_stuff(pdb_filepath):
    resfile = read_mutations_resfile("/Users/jameslucas/Kortemme_Rotation/"+pdb_filepath)
    for i in resfile:
        position = i[0]
        chain = i[1]
        mut_to = i[3]
    return position, chain, mut_to
    
#Prints CMD input with PDBID, associated mutation, and pivot residues
def bash(chaintomove, pdb_file):
    #Working directory
    workingdir = '/Users/jameslucas/Kortemme_Rotation/output/'
    #Removes PDB file from path, saves in variable data_dir
    pdb_file_parse = re.sub("/",' ', str(pdb_file))
    data, filenum, pdbtemp = pdb_file_parse.split()
    data_dir = data + "/" + filenum
    PDBID = pdbtemp[:-4]
    
    #Makes a folder for data dumping
    os.mkdir(workingdir + filenum)
    
    #Dictionary: 1- to 3-letter code
    res_dict = {
        'A':'ALA',
        'C':'CYS',
        'D':'ASP',
        'E':'GLU',
        'F':'PHE',
        'G':'GLY',
        'H':'HIS',
        'I':'ILE',
        'K':'LYS',
        'L':'LEU',
        'M':'MET',
        'N':'ASN',
        'P':'PRO',
        'Q':'GLN',
        'R':'ARG',
        'S':'SER',
        'T':'THR',
        'V':'VAL',
        'W':'TRP',
        'Y':'TYR'
    }
        
    #Assigns function output to variables for bash input (pivot_residues, target, new_res)
    target, chain, new_res_one = resfile_stuff(data_dir)
    new_res_three = res_dict[new_res_one]
    pivot_residues = neighbors_list(data_dir, pdb_file)
    
####All the variables and stuff for printing out the bash script
    
#Local Testing - All things

    arg = ['/Users/jameslucas/rosetta_src_2016.02.58402_bundle/main/source/bin/rosetta_scripts.macosclangrelease',
           '-s',
           '/Users/jameslucas/Kortemme_Rotation/%s/%s.pdb' %(data_dir, PDBID),
           '-parser:protocol',
           '/Users/jameslucas/Kortemme_Rotation/DDG_Test.xml',
           '-ignore_unrecognized_res',
           '-out:path:pdb',
           workingdir + filenum,
           '-parser:script_vars',
           'target=%s' %(target),
           'new_res=%s' %(new_res_three),
           'pivot_residues=%s' %(pivot_residues),
           '-nstruct 5'
          ]
    
    outfile_path = os.path.join(workingdir, 'rosetta.out')
    rosetta_outfile = open(outfile_path, 'w')
    rosetta_process = subprocess.Popen(arg, stdout=rosetta_outfile, cwd=workingdir)
    rosetta_outfile.close()

#    subprocess.call(arg)
        
chaintomove, pdb_file = json_parser()
bash(chaintomove, pdb_file)

In [ ]:
import os
import json
import re

asdf = open("/Users/jameslucas/Kortemme_Rotation/blank_job_dict.json")
jsonfile = json.load(asdf)

for i in jsonfile:
    print i

for i in jsonfile:
    a,b,filename = re.sub("/",' ', str(jsonfile[key]['input_file_list'][0])).split()
    print filename[:-4]

In [ ]:
#Iterates through directories to open Rosetta.out and creates Nested Dictionaries:
#>Prediction ID
#>>Structure ID (nstruct 1-100)
#>>>Structure Type (WT, Mut, DDG)
#>>>>Score Type

import os
import linecache

def parse_rosetta_out(workingdir):
    fattydict = {}
    for i in os.listdir(workingdir):
        if os.path.isdir(workingdir + i):
            fattydict[i] = {}
            structID = 1
            counter = 0
            filename = workingdir + i + "/rosetta.out"
            for line in enumerate(open(filename, 'r')):
                if line[1].find("fa_atr") == 1:
                    if counter % 2 == 0:
                        linecounter = 0
                        currentline = line[0] + 1
                        fattydict[i]['WT_' + str(structID)] = {}
                        while linecounter < 15:
                            scores = linecache.getline(filename, currentline)
                            parsed_scores = scores.split()
                            fattydict[i]['WT_' + str(structID)][parsed_scores[0]] = parsed_scores[1]
                            linecounter = linecounter + 1
                            currentline = currentline + 1
                        sumscore = linecache.getline(filename, line[0] + 17)
                        parsed_sumscore = sumscore.split()
                        fattydict[i]['WT_' + str(structID)][parsed_sumscore[0]] = parsed_sumscore[2]
                        counter = counter + 1
                    else:
                        linecounter = 0
                        currentline = line[0] + 1
                        fattydict[i]['Mutant_' + str(structID)] = {}
                        while linecounter < 15:
                            scores = linecache.getline(filename, currentline)
                            parsed_scores = scores.split()
                            fattydict[i]['Mutant_' + str(structID)][parsed_scores[0]] = parsed_scores[1]
                            linecounter = linecounter + 1
                            currentline = currentline + 1
                        sumscore = linecache.getline(filename, line[0] + 17)
                        parsed_sumscore = sumscore.split()
                        fattydict[i]['Mutant_' + str(structID)][parsed_sumscore[0]] = parsed_sumscore[2]
                        counter = counter + 1
                        structID = structID +1
                else:
                    continue
    return fattydict

my_working_directory = '/Users/jameslucas/Kortemme_Rotation/output/'
parsed_dict = parse_rosetta_out(my_working_directory)
print parsed_dict

In [ ]:
#qsub commands
qsub
qhold (job #)
qrls (job #).(task #'s):1